In [ ]:
import pandas as pd
import numpy as np
import torch
import os


In [ ]:
# tic_list = [tic[:-4] for tic in os.listdir("./CSI")]
# for i in range(len(tic_list)):
#     data = pd.read_csv(f"./CSI/{tic_list[i]}.csv", index_col=0)
#     os.remove(f"./CSI/{tic_list[i]}.csv")
#     if tic_list[i][-2:] == "SS":
#         mark = "SH"
#     else:
#         mark = "SZ"
#     tic_list[i] = mark + tic_list[i][:6]
#     data.to_csv(f"./CSI/{tic_list[i]}.csv")
   

In [ ]:
import qlib
from qlib.config import REG_CN
from qlib.contrib.data.handler import Alpha158

qlib.init(provider_uri="~/.qlib/CSI_data", region=REG_CN)
data_handler_config = {
    "start_time": "2010-01-01",
    "end_time": "2022-06-01",
    "instruments": "all",
}
h = Alpha158(**data_handler_config)
data_df = h.fetch()


In [ ]:
feature_df = data_df.reset_index().dropna(axis=1, how="any").rename(
    columns={"datetime": "date", "instrument": "tic"}
)
feature_df


In [ ]:
for k,v in feature_df.isna().sum().items():
    if v > 0:
        print(k, v)

In [ ]:
drop_col = [
    "ROC10",
    "ROC20",
    "ROC30",
    "ROC60",
    "RSQR5",
    "RSQR10",
    "RSQR20",
    "RSQR30",
    "RSQR60",
    "CORR5",
    "CORR10",
    "CORR20",
    "CORR30",
    "CORR60",
    "CORD10",
    "CORD20",
    "CORD30",
    "CORD60"
]


In [ ]:
feature_df = feature_df.drop(columns=drop_col)

In [ ]:
feature_df = feature_df.dropna()

In [ ]:
for tic in feature_df['tic'].unique():
    print(len(feature_df[feature_df['tic'] == tic]))

In [ ]:
alpha158 = feature_df.columns[2:].to_list() # alpha158
basic_feature = ["open", "close", "high", "low", "volume"]
target = "return+5"

In [ ]:
tic_df_list = []
for tic_path in os.listdir("./CSI/"):
    tic_df = pd.read_csv(f"./CSI/{tic_path}", index_col=0)[["date", "open", "close", "high", "low", "volume"]]
    tic_df["tic"] = tic_path[:8]
    tic_df["return+5"] = tic_df.close.diff(5).shift(-5) / tic_df.close
    tic_df_list.append(tic_df)

tic_target_df = pd.concat(tic_df_list).sort_values(by="date").dropna()
tic_target_df.date = pd.DatetimeIndex(tic_target_df.date)


In [ ]:
tic_target_df

In [ ]:
dataset_df = feature_df.merge(tic_target_df, how='inner', on=["date", "tic"])
dataset_df

In [ ]:
time_span = 60

In [ ]:
dataset_feat = []
dataset_ret = []

for tic in dataset_df.tic.unique():
    df = dataset_df[dataset_df.tic == tic]
    feat = df[basic_feature].to_numpy()
    ret = df[target].to_numpy()
    stock_feat = []
    stock_ret = []
    for i in range(time_span, feat.shape[0]):
        stock_feat.append(feat[i-time_span : i])
        stock_ret.append(ret[i])
    stock_feat = np.array(stock_feat)
    stock_ret = np.array(stock_ret)

    dataset_feat.append(stock_feat)
    dataset_ret.append(stock_ret)

dataset_feat = np.array(dataset_feat)
dataset_ret = np.array(dataset_ret)
        


In [ ]:
dataset_ret.shape

In [ ]:
dataset_ret = dataset_ret.transpose((1, 0))

In [ ]:
dataset_feat = dataset_feat.transpose((1, 2, 0, 3))

In [ ]:
import torch

dataset_ret_tensor = torch.tensor(dataset_ret, dtype=torch.float)
dataset_feat_tensor = torch.tensor(dataset_feat, dtype=torch.float)

In [ ]:
torch.save(dataset_ret_tensor, "./dataset/basic_feat/returns.pt")
torch.save(dataset_feat_tensor, "./dataset/basic_feat/features.pt")